In [72]:
import pandas as pd
import psycopg2

In [73]:
server = 'localhost'
database = 'python_etl'
user = 'postgres'
password = 'root'

connection_string = f"dbname={database} user={user} password={password} host={server}"

try:
    conn = psycopg2.connect(connection_string)
    print("Conexão bem-sucedida ao PostgreSQL!")
    
    cursor = conn.cursor()
except Exception as e:
    print("Erro ao conectar ao PostgreSQL:", e)

Conexão bem-sucedida ao PostgreSQL!


In [74]:
dados = pd.read_excel("/home/juniorsilvadev/engenharia-dados/python-etl/Origem/Origem/arquivos_excel/items.xlsx")

In [75]:
str(dados.columns).replace("'", "")

'Index([id, order_id, product_id, quantity, total_price], dtype=object)'

In [77]:
print(dados.dtypes)

id               int64
order_id         int64
product_id       int64
quantity         int64
total_price    float64
dtype: object


In [78]:
create_table_query = """
CREATE TABLE IF NOT EXISTS items (
    id SERIAL PRIMARY KEY,
    order_id INTEGER,
    product_id INTEGER,
    quantity INTEGER,
    total_price NUMERIC(10, 2)
);
"""

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Tabela 'items' criada com sucesso!")
except Exception as e:
    print("Erro ao criar a tabela 'categorias':", e)
    conn.rollback()

Tabela 'items' criada com sucesso!


In [79]:
cursor.execute("TRUNCATE TABLE items")

In [80]:
insert_query = """
INSERT INTO items (id, order_id, product_id, quantity, total_price)
VALUES (%s, %s, %s, %s, %s)
"""

try:
    if conn.closed:
        conn = psycopg2.connect(connection_string)
    
    cursor = conn.cursor()
    
    for index, row in dados.iterrows():
        cursor.execute(insert_query, (
            int(row['id']), 
            int(row['order_id']), 
            int(row['product_id']), 
            int(row['quantity']), 
            float(row['total_price'])
        ))
    
    conn.commit()
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Erro ao inserir dados:", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()

Dados inseridos com sucesso!
